In [1]:
import pandas as pd
import numpy as np 

In [3]:
train_features = pd.read_csv('../../Data/KAERI/train_features.csv')
train_target = pd.read_csv('../../Data/KAERI/train_target.csv', index_col = 'id')
test_features = pd.read_csv('../../Data/KAERI/test_features.csv')

In [4]:
def preprocessing_KAERI(data) :
    '''
    data: train_features.csv or test_features.csv
    
    return: Random Forest 모델 입력용 데이터
    '''
    
    # 충돌체 별로 0.000116 초 까지의 가속도 데이터만 활용해보기 
    _data = data.groupby('id').head(30)
    
    # string 형태로 변환
    _data['Time'] = _data['Time'].astype('str')
    
    # Random Forest 모델에 입력 할 수 있는 1차원 형태로 가속도 데이터 변환
    _data = _data.pivot_table(index = 'id', columns = 'Time', values = ['S1', 'S2', 'S3', 'S4'])
    
    # column 명 변환
    _data.columns = ['_'.join(col) for col in _data.columns.values]
    
    return _data

In [5]:
train_features = preprocessing_KAERI(train_features)
test_features = preprocessing_KAERI(test_features)

C:\Users\lonj7\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [6]:
import sklearn
from sklearn.ensemble import RandomForestRegressor

In [14]:
model = RandomForestRegressor(n_jobs=-1, random_state=23)

In [15]:
%%time
model.fit(train_features, train_target)

Wall time: 2.07 s


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=23, verbose=0, warm_start=False)

In [16]:
y_pred = model.predict(test_features)

In [17]:
num = np.arange(2800, 3500, 1)

In [18]:
submit = pd.DataFrame({'id': num, 'X': y_pred[:, 0], 'Y': y_pred[:, 1], 'M': y_pred[:, 2], 'V': y_pred[:, 3]})

In [20]:
submit.to_csv('submission/03.csv', index = False)